# 管道的内部 (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [4]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis",
                      model="distilbert-base-uncased-finetuned-sst-2-english"
                      )
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

pipline内部包含前处理、model处理、后处理三个过程

（1）第一步，用Tokenizer将输入转成模型可处理的格式

In [5]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, 
                   return_tensors="pt" # 根据设置的框架返回张量类型
                   )
print(inputs)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


（2）第二步，加载模型，输入经过模型得到输出。

In [8]:
# AutoModel可以得到输入经过模型处理后的特征表示
from transformers import AutoModel

model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)  # Transformer模型的输出是namedtuple或字典对象，outputs["last_hidden_state"]得到隐状态张量
print(outputs.last_hidden_state.shape)  # 批尺寸，序列长度和隐变量大小

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 16, 768])


在Transformers库中，有许多模型架构，他们一般有基础Transformer架构加上不同的head模块组成，例如：<br>
*Model (retrieve the hidden states)：只输出隐状态<br>
*ForCausalLM：常规语言模型，典型的有GPT系列<br>
*ForMaskedLM：掩码语言模型，典型的有BERT、RoBERTa、DeBERTa<br>
*ForMultipleChoice：多项选择模型<br>
*ForQuestionAnswering：问答模型，一般是抽取式问答<br>
*ForSequenceClassification：序列分类模型<br>
*ForTokenClassification：token分类模型，如命名实体识别和关系抽取<br>

In [6]:
# AutoModelForSequenceClassification进行序列分类，得到分类得分
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"  # 情感分类（二分类）
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

In [ ]:
print(outputs.logits.shape)  # 【batchsize,nclass】

torch.Size([2, 2])

In [ ]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward>)

（3）第三步，后处理模型输出，得到概率值

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

In [ ]:
# 分类标签
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}